In [5]:
# Use this notebook to extract specified chords from IDMT-SMT-Chords dataset
# IDMT-SMT-Chords Dataset: https://www.idmt.fraunhofer.de/en/publications/datasets/chords.html
# Download at https://zenodo.org/records/7544213

from IPython.display import display, Audio
from pydub import AudioSegment
import numpy as np  
import os

In [6]:
# Configurations *Change accordingly*

chords = ["A:maj", "A:min", "B:maj", "B:min", "C:maj", "C:min", "D:maj", "D:min", "E:maj", "E:min", "F:maj", "F:min", "G:maj", "G:min"] # possible chords, in <root:shorthand> format (e.g. C:maj, E:min, etc., check _readme.txt in dataset folder)
dataset_root = "./datasets/idmt-smt-chords/" # where you unzip the whole dataset
sound_setting_index = 0 # choose how your guitar sounds like (Check POSSIBLE_SOUND_SETTINGS)
output_dir = "./chords/" # make sure that it exists

In [7]:
# Constants & Functions *Do not change*

POSSIBLE_SOUND_SETTINGS = [
    "garageband_guitar_Acoustic_Guitar", #0
    "ableton_live_guitar_Nylon_Concerto_Guitar",
    "ableton_live_guitar_Campfire",
    "garageband_guitar_Hard_Rock",
    "ableton_live_guitar_Guitar_Open",
    "garageband_guitar_Classic_Clean"
    ]

WAV_FILE_PATH = os.path.join(dataset_root, "guitar", POSSIBLE_SOUND_SETTINGS[sound_setting_index] + ".wav")
ANNOTATION_FILE_PATH = os.path.join(dataset_root, "guitar", "guitar_annotation.lab")

def crop_wav(file_path, start_time, end_time):
    audio = AudioSegment.from_wav(file_path)
    return audio[start_time*1000:end_time*1000]

def crop_wav_to_numpy(file_path, start_time, end_time):
    cropped_audio = crop_wav(file_path, start_time, end_time)
    
    samples = np.array(cropped_audio.get_array_of_samples())
    
    if cropped_audio.channels == 2:
        samples = samples.reshape((-1, 2))
    
    return samples, cropped_audio.frame_rate

def play_numpy_array(samples, sample_rate):
    return Audio(samples, rate=sample_rate)

def read_chord_annotations(file_path):
    annotations = []
    
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split('\t')
            if len(parts) == 3:
                start_time, end_time, chord = parts
                annotations.append({
                    'start_time': float(start_time),
                    'end_time': float(end_time),
                    'chord': chord
                })
    
    return annotations

ANNOTATIONS = read_chord_annotations(ANNOTATION_FILE_PATH)


def find_chord_annotation_lowest_pitch(chord):
    for annotation in ANNOTATIONS:
        if annotation['chord'] == chord:
            return annotation
    raise ValueError(f"Chord {chord} not found in annotations")

def get_chord_numpy_lowest_pitch(chord):
    annotation = find_chord_annotation_lowest_pitch(chord)
    pressure_array, sample_rate = crop_wav_to_numpy(WAV_FILE_PATH, annotation["start_time"], annotation["end_time"])
    return pressure_array, sample_rate

def crop_and_save_chord_lowest_pitch(chord):
    annotation = find_chord_annotation_lowest_pitch(chord)
    cropped_audio = crop_wav(WAV_FILE_PATH, annotation["start_time"], annotation["end_time"])
    cropped_audio.export(os.path.join(output_dir, chord + ".wav"), format="wav")

In [8]:
# Try out segment for C Major chord
arr, rate = get_chord_numpy_lowest_pitch("D:min")
audio_widget = play_numpy_array(arr, rate)
display(audio_widget)

In [9]:
# Crop out specified chords and save to output directory
for chord in chords:
    crop_and_save_chord_lowest_pitch(chord)